# Swisscom Speech Assignment
## Gender classification using MFCC

Gaétan Ramet

In [1]:
# Basic libraries
import os, sys
import numpy as np
import scripts.lib as lib

# Speech
import soundfile as sf # pip install pysoundfile
import python_speech_features as speech_lib # pip install python_speech_features

# Machine learning
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from sklearn.metrics import classification_report


%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4)


The goal of this assignment is to extract the MFCC features from the audio files of the the 'dev-clean' dataset and use them to predict the gender of speakers. In this notebook, we will first extract the features from the raw data and arrange them in a dataset. We will then experiment with different types of classifiers to assess their performances

## 1. Generate dataset from raw data

First, we will load the metadata of 'dev-clean' from the 'SPEAKERS.TXT' file to have the gender and ids of the different speakers. We don't need the other information from 'SPEAKERS.TXT'. We will assign the value '0' to the female speakers class and the value '1' to the male speakers class

In [2]:
speakers_file = './dev-clean/LibriSpeech/SPEAKERS.TXT'

with open(speakers_file) as f:
    content = f.readlines() 

id_speaker = np.array([], dtype=int)
gender_speaker = np.array([], dtype=int)
for line in content:
    if 'dev-clean' in line:
        id_speaker = np.append(id_speaker, int(line.split('|')[0]))
        if 'F' in line.split('|')[1]:
            gender_speaker = np.append(gender_speaker,0)
        elif 'M' in line.split('|')[1]:
            gender_speaker = np.append(gender_speaker,1)

gender_speaker = np.asarray(gender_speaker)
id_speaker = np.asarray(id_speaker)
metadata = np.concatenate([np.expand_dims(id_speaker, axis=1).T, np.expand_dims(gender_speaker,axis=1).T]).T

for row  in metadata:
    print("Speaker id : {:5d}, gender class : {:d}".format(row[0],row[1]))

Speaker id :    84, gender class : 0
Speaker id :   174, gender class : 1
Speaker id :   251, gender class : 1
Speaker id :   422, gender class : 1
Speaker id :   652, gender class : 1
Speaker id :   777, gender class : 1
Speaker id :  1272, gender class : 1
Speaker id :  1462, gender class : 0
Speaker id :  1673, gender class : 0
Speaker id :  1919, gender class : 0
Speaker id :  1988, gender class : 0
Speaker id :  1993, gender class : 0
Speaker id :  2035, gender class : 0
Speaker id :  2078, gender class : 1
Speaker id :  2086, gender class : 1
Speaker id :  2277, gender class : 0
Speaker id :  2412, gender class : 0
Speaker id :  2428, gender class : 1
Speaker id :  2803, gender class : 1
Speaker id :  2902, gender class : 1
Speaker id :  3000, gender class : 1
Speaker id :  3081, gender class : 0
Speaker id :  3170, gender class : 1
Speaker id :  3536, gender class : 0
Speaker id :  3576, gender class : 0
Speaker id :  3752, gender class : 1
Speaker id :  3853, gender class : 0
S

To have a fair prediction, we don't want to train and test our models on the same speakers. We will then split the dataset into a training and a testing set, while keeping the 50/50 ratio of male and female speakers in each set

In [3]:
female_speakers = metadata[metadata[:,1]==0]
male_speakers = metadata[metadata[:,1]==1]

train_female, test_female = lib.split_train_test_speakers(female_speakers[:,0],ratio = 0.5, seed=1)
train_male, test_male = lib.split_train_test_speakers(male_speakers[:,0],ratio = 0.5, seed=0)

train_speakers = np.append(np.append(np.expand_dims(train_female,axis=1),np.zeros([train_female.shape[0],1]),axis=1),
                           np.append(np.expand_dims(train_male,axis=1),np.ones([train_male.shape[0],1]),axis=1),axis=0).astype(int)
test_speakers = np.append(np.append(np.expand_dims(test_female,axis=1),np.zeros([test_female.shape[0],1]),axis=1),
                           np.append(np.expand_dims(test_male,axis=1),np.ones([test_male.shape[0],1]),axis=1),axis=0).astype(int)

print("Training set : ")
for row  in train_speakers:
    print("Speaker id : {:5d}, gender class : {:d}".format(row[0],row[1]))
    
print("Testing set : ")
for row  in test_speakers:
    print("Speaker id : {:5d}, gender class : {:d}".format(row[0],row[1]))

Training set : 
Speaker id :  1919, gender class : 0
Speaker id :  6319, gender class : 0
Speaker id :  2035, gender class : 0
Speaker id :  3536, gender class : 0
Speaker id :  1673, gender class : 0
Speaker id :  5895, gender class : 0
Speaker id :  1988, gender class : 0
Speaker id :  6345, gender class : 0
Speaker id :  2277, gender class : 0
Speaker id :  1462, gender class : 0
Speaker id :  7976, gender class : 1
Speaker id :   251, gender class : 1
Speaker id :  8297, gender class : 1
Speaker id :  2428, gender class : 1
Speaker id :  2902, gender class : 1
Speaker id :  6295, gender class : 1
Speaker id :  2078, gender class : 1
Speaker id :  3752, gender class : 1
Speaker id :   777, gender class : 1
Speaker id :   422, gender class : 1
Testing set : 
Speaker id :  5338, gender class : 0
Speaker id :    84, gender class : 0
Speaker id :  8842, gender class : 0
Speaker id :  7850, gender class : 0
Speaker id :  3081, gender class : 0
Speaker id :  6313, gender class : 0
Speaker

Now that we have our train and test speakers with balanced male/female repartition, we will extract the mean MFCCs of each audio file to create our training and testing data as well as the ground truth table.  

MFCCs are computed using the [python_speech_features library](https://github.com/jameslyons/python_speech_features "Python_speech_features"). We used a window length of 60 ms, so that it is long enough to cover a few periods of the speech signal. We decided to compute 40 MFCCs, which is arbitrary but gives us decent classification results

In [4]:
mfcc_num = 40

train_data, train_gender = lib.create_dataset(train_speakers, mfcc_num=mfcc_num)
test_data, test_gender = lib.create_dataset(test_speakers, mfcc_num=mfcc_num)

print(train_data.shape)
print(train_gender.shape)

loading data from speaker  1919
loading data from speaker  6319
loading data from speaker  2035
loading data from speaker  3536
loading data from speaker  1673
loading data from speaker  5895
loading data from speaker  1988
loading data from speaker  6345
loading data from speaker  2277
loading data from speaker  1462
loading data from speaker  7976
loading data from speaker  251
loading data from speaker  8297
loading data from speaker  2428
loading data from speaker  2902
loading data from speaker  6295
loading data from speaker  2078
loading data from speaker  3752
loading data from speaker  777
loading data from speaker  422
loading data from speaker  5338
loading data from speaker  84
loading data from speaker  8842
loading data from speaker  7850
loading data from speaker  3081
loading data from speaker  6313
loading data from speaker  2412
loading data from speaker  3853
loading data from speaker  3576
loading data from speaker  1993
loading data from speaker  1272
loading data 

## 2. Classifiers

We will here experiment with different classifiers and assess their performances. But first, let us normalize the data by subtracting the mean and dividing by the standard deviation. The first 4 classifiers are implemented using the [sklearn library](http://scikit-learn.org/stable/ "scikit-learn") and the CNN is written with [Tensorflow](https://www.tensorflow.org/ "Tensorflow")

In [5]:
train_data, test_data = lib.normalize_data(train_data, test_data)

### 2.1. Gaussian Naive Bayes classifier

The Gaussian Naive Bayes classifier makes the assumption that the features are independant and that each feature is normally distributed. It is a simple classifier and is very fast as it can be solved in closed form, unlike iterative models presented below. 

In [6]:
clf = GaussianNB()
clf.fit(train_data,np.squeeze(train_gender))

train_accuracy = clf.score(train_data,np.squeeze(train_gender))
print("Training Accuracy : ", train_accuracy)
lib.test_classifier(clf,test_data, np.squeeze(test_gender))

Training Accuracy :  0.916256157635
Test accuracy :  0.9375975039
26 males classified as females out of 631, 4.120 %
54 females classified as males out of 651, 8.295 %
             precision    recall  f1-score   support

        0.0       0.96      0.92      0.94       651
        1.0       0.92      0.96      0.94       631

avg / total       0.94      0.94      0.94      1282



### 2.2. K-NN classifier

The K-Nearest Neighbors classifier is based on a "vote" of the K data points that are the closest (in L2 norm) to the test sample. Defining the K number of neighbors that are taken into account can be done empirically, as presented here:

In [7]:
best_acc = 0
max_nn = 101
for num_neighbors in np.linspace(1,max_nn,np.ceil(max_nn/2).astype(int)):

    clf = KNeighborsClassifier(n_neighbors=int(num_neighbors))
    clf.fit(train_data,np.squeeze(train_gender))

    accuracy = clf.score(test_data,test_gender)
    if accuracy > best_acc:
        best_acc = accuracy
        best_nn = int(num_neighbors)
        print("New optimal parameters : number of neighbors : ", best_nn, ", accuracy : ", best_acc)

New optimal parameters : number of neighbors :  1 , accuracy :  0.840093603744
New optimal parameters : number of neighbors :  3 , accuracy :  0.867394695788
New optimal parameters : number of neighbors :  5 , accuracy :  0.874414976599
New optimal parameters : number of neighbors :  7 , accuracy :  0.879875195008
New optimal parameters : number of neighbors :  9 , accuracy :  0.883775351014
New optimal parameters : number of neighbors :  13 , accuracy :  0.886895475819
New optimal parameters : number of neighbors :  31 , accuracy :  0.88767550702
New optimal parameters : number of neighbors :  47 , accuracy :  0.890015600624
New optimal parameters : number of neighbors :  49 , accuracy :  0.891575663027
New optimal parameters : number of neighbors :  53 , accuracy :  0.893135725429
New optimal parameters : number of neighbors :  59 , accuracy :  0.89391575663
New optimal parameters : number of neighbors :  69 , accuracy :  0.894695787832
New optimal parameters : number of neighbors : 

In [8]:
clf = KNeighborsClassifier(n_neighbors=best_nn)
clf.fit(train_data,np.squeeze(train_gender))

train_accuracy = clf.score(train_data,np.squeeze(train_gender))
print("Training Accuracy : ", train_accuracy)
lib.test_classifier(clf,test_data, np.squeeze(test_gender))

Training Accuracy :  0.96833216045
Test accuracy :  0.897035881435
66 males classified as females out of 631, 10.460 %
66 females classified as males out of 651, 10.138 %
             precision    recall  f1-score   support

        0.0       0.90      0.90      0.90       651
        1.0       0.90      0.90      0.90       631

avg / total       0.90      0.90      0.90      1282



### 2.3. SVM

Support Vector Machine (SVM) is a classification method that tries to separate the data with decision lines. It is a maximum margin method, which means it tries to maximize the distance between the classes at the decision line. SVM performs a linear classification, but it can be extended to non-linear classification by using a Kernel transformation (here, we used a polynomial kernel)

In [9]:
best_acc=0
for degree in range(1,15):

    clf = svm.SVC(degree=degree,kernel='poly')
    clf.fit(train_data,np.squeeze(train_gender))

    accuracy = clf.score(test_data,test_gender)
    
    if accuracy > best_acc:
        best_acc = accuracy
        best_degree = degree
    
        print("New optimal parameters : degree : ", best_degree, ", accuracy : ", accuracy)



New optimal parameters : degree :  1 , accuracy :  0.833073322933
New optimal parameters : degree :  3 , accuracy :  0.873634945398


In [10]:
clf = svm.SVC(degree=best_degree, kernel='poly')

clf.fit(train_data,np.squeeze(train_gender))

train_accuracy = clf.score(train_data,np.squeeze(train_gender))
print("Training Accuracy : ", train_accuracy)
lib.test_classifier(clf,test_data, np.squeeze(test_gender))

Training Accuracy :  0.999296270232
Test accuracy :  0.873634945398
108 males classified as females out of 631, 17.116 %
54 females classified as males out of 651, 8.295 %
             precision    recall  f1-score   support

        0.0       0.85      0.92      0.88       651
        1.0       0.91      0.83      0.87       631

avg / total       0.88      0.87      0.87      1282



### 2.4. Neural Network (Multi-Layer Perceptron)


The Multi-layer Perceptron is a Neural Network that is able to classify non linearly separable data. It trains iteratively and uses backpropagation to learn the data. 

Here, we will use a Perceptron with: one input layer of size mfcc_num+1, two hidden layers and one output layer of size 2 since we have two classes. The "optimal" size of the two hidden layers is empirically determined based on the testing accuracy:

In [11]:
best_acc = 0
for hidden_size in range(2,mfcc_num+1):
    for hidden_size2 in range(2,hidden_size):

        clf = MLPClassifier(solver='lbfgs', alpha=1e-1,
                    hidden_layer_sizes=(hidden_size, hidden_size2), random_state=1)

        clf.fit(train_data, np.squeeze(train_gender))

        accuracy = clf.score(test_data,test_gender)
        if accuracy > best_acc:
            best_acc = accuracy
            best_hs1 = hidden_size
            best_hs2 = hidden_size2
            print(" New optimal parameters : Hidden layer sizes : ", best_hs1, best_hs2,", accuracy : ", best_acc)

 New optimal parameters : Hidden layer sizes :  3 2 , accuracy :  0.834633385335
 New optimal parameters : Hidden layer sizes :  4 2 , accuracy :  0.854134165367
 New optimal parameters : Hidden layer sizes :  4 3 , accuracy :  0.866614664587
 New optimal parameters : Hidden layer sizes :  6 3 , accuracy :  0.886895475819
 New optimal parameters : Hidden layer sizes :  8 3 , accuracy :  0.895475819033
 New optimal parameters : Hidden layer sizes :  8 4 , accuracy :  0.896255850234
 New optimal parameters : Hidden layer sizes :  11 4 , accuracy :  0.90639625585
 New optimal parameters : Hidden layer sizes :  13 2 , accuracy :  0.913416536661
 New optimal parameters : Hidden layer sizes :  15 5 , accuracy :  0.918096723869


And now we re-run the Perceptron with the best parameters:

In [12]:
clf =  MLPClassifier(solver='lbfgs', alpha=1e-1,
                    hidden_layer_sizes=(best_hs1, best_hs2), random_state=1)
clf.fit(train_data,np.squeeze(train_gender))

train_accuracy = clf.score(train_data,np.squeeze(train_gender))
print("Training Accuracy : ", train_accuracy)
lib.test_classifier(clf,test_data, np.squeeze(test_gender))

Training Accuracy :  1.0
Test accuracy :  0.918096723869
55 males classified as females out of 631, 8.716 %
50 females classified as males out of 651, 7.680 %
             precision    recall  f1-score   support

        0.0       0.92      0.92      0.92       651
        1.0       0.92      0.91      0.92       631

avg / total       0.92      0.92      0.92      1282



### 2.5. Deep learning

Here we will experiment with a simple CNN written in Tensorflow. Our CNN is composed of a convolutional layer (using 40 1D filters of length 9, as our feature vector is 1D), followed by a fully connected layer and a softmax for classification. To avoid overfitting during the training, we used dropout on the FC layer and L2 regularizaion over the entire network. 

We start training on 50 epochs with a batch size of 128 samples and use an early stopping criterium based on the increase of the test error, i.e. the beginning of overfitting. On average, the training stops after 3-10 epochs, with a test accuracy of 92-95%

In [13]:
learning_rate = 0.001
reg = 1e-2
n_filters = 40
kernel_size = 9

tf_data = tf.placeholder(tf.float32,[None, mfcc_num,1])
tf_labels = tf.placeholder(tf.float32,[None, 2])
keep_prob = tf.placeholder(tf.float32)

regularizer = tf.contrib.layers.l2_regularizer(scale=reg)

#Conv layer
conv_out = tf.layers.conv1d(inputs=tf_data, filters=n_filters, kernel_size=kernel_size, kernel_regularizer=regularizer, bias_regularizer=regularizer)

#FC layer
fc_out = tf.contrib.layers.fully_connected(inputs=tf.reshape(conv_out,[-1, (mfcc_num-kernel_size+1)*n_filters]), num_outputs=2, weights_regularizer=regularizer, biases_regularizer=regularizer)
fc_out = tf.layers.dropout(inputs=fc_out,rate=keep_prob)

#Softmax
softmax_out = tf.nn.softmax(fc_out)

#loss
cross_entropy = tf.losses.softmax_cross_entropy(tf_labels, softmax_out)

#Regularization
reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term = tf.contrib.layers.apply_regularization(regularizer, reg_variables)
loss = reg_term + cross_entropy

#Optimizer
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(softmax_out,axis=1), tf.argmax(tf_labels,axis=1)), tf.float32))

In [21]:
n_epoch = 50
batch_size = 128
dropout = 0.7

train_gender_tf = lib.one_hot_convert(np.squeeze(train_gender.astype(int)))
test_gender_tf = lib.one_hot_convert(np.squeeze(test_gender.astype(int)))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

best_acc = 0
for epoch in range(n_epoch):
    indices = np.random.permutation(train_data.shape[0])
    for iter_ in range(int(np.ceil(train_data.shape[0]/batch_size))):
        
        batch_idx = indices[iter_*batch_size:min((iter_+1)*batch_size, indices.shape[0])]
        batch_x = np.expand_dims(train_data[batch_idx,:],axis=2)
        batch_y = train_gender_tf[batch_idx,:]
        
        _, train_acc, train_loss = sess.run([train_step, accuracy, loss],feed_dict={tf_data : batch_x, tf_labels : batch_y, keep_prob : dropout})

    if epoch%1==0:

        _, train_acc, train_loss = sess.run([softmax_out, accuracy, loss],feed_dict={tf_data : np.expand_dims(train_data,axis=2), tf_labels : train_gender_tf, keep_prob : 1})
        pred, test_acc, test_loss = sess.run([softmax_out, accuracy, loss], feed_dict={tf_data : np.expand_dims(test_data,axis=2), tf_labels : test_gender_tf, keep_prob:1} )   
        
        print("epoch ", epoch+1,", test accuracy : ", test_acc, ", train accuracy : ", train_acc)
       
        pred = np.argmax(np.round(np.asarray(pred)),axis=1)
        if test_acc > best_acc:
            best_acc = test_acc
            report = classification_report(np.squeeze(test_gender),pred)
        else:
            print("Early stoping, test accuracy is decreasing")
            print(" Best Test accuracy : ", best_acc)
            print(report)
            break
    


epoch  1 , test accuracy :  0.943838 , train accuracy :  0.904293
epoch  2 , test accuracy :  0.949298 , train accuracy :  0.908515
epoch  3 , test accuracy :  0.946178 , train accuracy :  0.913441
Early stoping, test accuracy is decreasing
 Best Test accuracy :  0.949298
             precision    recall  f1-score   support

        0.0       0.97      0.93      0.95       651
        1.0       0.93      0.97      0.95       631

avg / total       0.95      0.95      0.95      1282



# 3. Discussion

From this assignment, we can see that complex models does not always mean better classification. In fact, our most simple classifier (Gaussian Naive bayes) outperforms most of the other classifiers we tried and come very close to the performances of a simple CNN. 

This can be partially explained by the data that is used for classification and training. Here, we classified each audio sample based on the mean MFCCs over the sample. However, since the audio are very short, and the classification mostly relies on the frequencies of the sound, samples from the same class (i.e. male or female) are going to be very widely spread apart depending on the content of the audio file. An audio with a lot of phonemes that are on higher frequencies is more likely to be classified as spoken by a female speaker simply because it contains those phonemes and not because of the speaker characteristics. To avoid this phenomenon, we could use longer audio samples or averaging the MFCCs over multiple audio samples. This would also allow to achieve mcuh higher accuracy... Another solution would be to train and classify using the same sentences in the audio files, so that the content is the same and only the speaker characteristics are taken into account, but then we need to use that specific sentence for classification.